https://github.com/davidrmiller/biosim4


In [1]:
from secrets import token_hex
import secrets
import random
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

decode hexadecimal

In [2]:
def split_genome(hexval):
    binary = bin(int(hexval, 16))[2:]
    if len(binary) < 32:
        factor = 32 - len(binary)
        binary = '0' * factor + binary
  
    source_type, source_id = binary[0], binary[1:8]
    sink_type, sink_id = binary[8], binary[9:16]
    weight_sign, weight = binary[16], binary[17:]

    gene_value_list = {key:int(val, 2) for key, val in zip(['source_type', 'source_id', 'sink_type', 'sink_id', 'weight_sign', 'weight'],
                                          [source_type, source_id, sink_type, sink_id, weight_sign, weight])}
    
    return gene_value_list

hexadecimal generator

In [3]:
nr_of_input = 13
nr_of_actions = 5
nr_of_inner = 1
nr_of_genes = 4

def generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner):
    lista = {}
    for nr_idividual in range(nr_individuals):
        gene_translated = {}
        hexa_list = [token_hex(4) for i in range(nr_of_genes)]
        for hexval in hexa_list:

            gen_component = split_genome(hexval)
            
            if gen_component['source_type'] == 0:
                input_id = [gen_component['source_id'] % nr_of_input, 'in']
            elif gen_component['source_type'] == 1:
                input_id = [gen_component['source_id'] % nr_of_inner, 'mid']
            
            if gen_component['sink_type'] == 0:
                output_id = [gen_component['sink_id'] % nr_of_inner, 'mid']
            elif gen_component['sink_type'] == 1:
                output_id = [gen_component['sink_id'] % nr_of_actions, 'out']

            if gen_component['weight_sign'] == 0:
                weight_val = (gen_component['weight'] /8191.25) * -1
            elif gen_component['weight_sign'] == 1:
                weight_val = gen_component['weight'] / 8191.25

            gene_translated[hexval] = {'input':input_id, 'weight':weight_val, 'output':output_id}
        lista[nr_idividual] = gene_translated
    return lista          

In [4]:
res = generate_initial_genomes_for_population(2, 4, nr_of_input, nr_of_actions, nr_of_inner)
res

{0: {'b05068dc': {'input': [0, 'mid'],
   'weight': -3.277155501297116,
   'output': [0, 'mid']},
  'a299194d': {'input': [0, 'mid'],
   'weight': -0.790721806806043,
   'output': [0, 'out']},
  '61922695': {'input': [6, 'in'],
   'weight': -1.205798870746223,
   'output': [3, 'out']},
  '503eb1a2': {'input': [2, 'in'],
   'weight': 1.5511674042423318,
   'output': [0, 'mid']}},
 1: {'4edee770': {'input': [0, 'in'],
   'weight': 3.2327178391576377,
   'output': [4, 'out']},
  '8f83055b': {'input': [0, 'mid'],
   'weight': -0.16737372195940792,
   'output': [3, 'out']},
  'd152833f': {'input': [0, 'mid'],
   'weight': 0.10144971768655578,
   'output': [0, 'mid']},
  '644b6221': {'input': [9, 'in'],
   'weight': -3.0668090950709597,
   'output': [0, 'mid']}}}

In [5]:
res[1]

{'4edee770': {'input': [0, 'in'],
  'weight': 3.2327178391576377,
  'output': [4, 'out']},
 '8f83055b': {'input': [0, 'mid'],
  'weight': -0.16737372195940792,
  'output': [3, 'out']},
 'd152833f': {'input': [0, 'mid'],
  'weight': 0.10144971768655578,
  'output': [0, 'mid']},
 '644b6221': {'input': [9, 'in'],
  'weight': -3.0668090950709597,
  'output': [0, 'mid']}}

In [6]:

for hexa in res[0]:
    print(res[0][hexa])

{'input': [0, 'mid'], 'weight': -3.277155501297116, 'output': [0, 'mid']}
{'input': [0, 'mid'], 'weight': -0.790721806806043, 'output': [0, 'out']}
{'input': [6, 'in'], 'weight': -1.205798870746223, 'output': [3, 'out']}
{'input': [2, 'in'], 'weight': 1.5511674042423318, 'output': [0, 'mid']}


### data model

In [7]:
obj_list = {0:{'coord': [[2,3]], 'neuron':
               {'e20775ad': {'input': [0, 'mid'],'weight': -3.6777048679993896, 'output': [0, 'mid']},
                'ef9aa29c': {'input': [0, 'mid'], 'weight': 1.0816419960323516, 'output': [1, 'out']},
                '9bdf3966': {'input': [0, 'mid'], 'weight': -1.793865405157943,'output': [0, 'out']},
                'a079dd71': {'input': [0, 'mid'], 'weight': 2.920311307797955,'output': [0, 'mid']}}},
            1:{'coord': [[2,3]], 'neuron':
              {'b18c8e5e': {'input': [0, 'mid'],'weight': 0.4490157179917595,'output': [2, 'out']},
               'b8f967c7': {'input': [0, 'mid'],  'weight': -3.2433389287349303, 'output': [1, 'out']},
               '673cf9a3': {'input': [12, 'in'], 'weight': 3.8014954982450786, 'output': [0, 'mid']},
               'c1a2706c': {'input': [0, 'mid'], 'weight': -3.51350526476423, 'output': [4, 'out']}}}}


## or

In [ ]:
class Creature():
    

### funkcje neuronów

input

In [8]:
# Odległość od granicy świata x 

# -5-5

# Odległość od granicy świata y

# -5-5

# Bliska przeszkoda

def close_obstacle(obj_loc_x, obj_loc_y, obj_list):
    select_obst = 0
    # iteruj po liście bez obiektu
    for obj_nr in obj_list:
        if obj_nr['coord'][0] - obj_loc_x == abs(1) and obj_nr['coord'][1] - obj_loc_y == abs(1):
            select_obst += 1
            
    return select_obst/8
       
# Daleka przeszkoda gdy ruch po osi x

def distant_obstacle_x_coord(obj_loc_x, obj_list):
    select_obst = 0
    # iteruj po liście bez obiektu
    for obj_nr in obj_list:
        if obj_nr['coord'][0] - obj_loc_x <= abs(5):
            select_obst += 1
            
    return select_obst/121

# Daleka przeszkoda gdy ruch po osi y

# to samo j.w.
def distant_obstacle_y_coord(obj_loc_y, obj_list):
    select_obst = 0
    # iteruj po liście bez obiektu
    for obj_nr in obj_list:
        if obj_nr['coord'][0] - obj_loc_y <= abs(5):
            select_obst += 1
            
    return select_obst/121


output

przykład jak zastosować neurony executeActions.cpp

In [9]:
def ruch_x(weight):
    weight_abs = abs(weight)
    x = np.random.choice(2, 1, p=[1-weight_abs, weight_abs])
    if weight < 0:
        x = x * -1
    return x

def ruch_y(weight):
    weight_abs = abs(weight)
    y = np.random.choice(2, 1, p=[1-weight_abs, weight_abs])
    if weight < 0:
        y = y * -1
    return y
   
# Ruch losowy

def random_move():
    x,y = np.round(np.random.uniform(low=-1, high=1.0, size=2))
    return x,y
    

generate coord with given p value

In [10]:
# prawdopodobieństwo wystąpienia 1

for i in range(10):
    print(np.random.choice(2, 1, p=[0.01,0.99]))
    print(np.random.choice(2, 1, p=[0.71,0.29]))


[1]
[0]
[1]
[0]
[1]
[1]
[1]
[0]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[0]
[1]
[0]
[1]
[0]
